In [ ]:
from langchain_aws.agents.base import BedrockAgentsRunnable
from langchain.schema.messages import AIMessage, HumanMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_aws.agents.base import BedrockAgentsRunnable
from langchain.agents import Tool, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from importlib import reload
import config as conf
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langsmith import traceable
import boto3

!export LANGCHAIN_TRACING_V2=true
!export LANGCHAIN_API_KEY=conf.langsmith_api

In [5]:
search = DuckDuckGoSearchResults()

In [7]:
region_name = 'us-east-1'
bedrock_agent = boto3.client(service_name = 'bedrock-agent-runtime', region_name = region_name)

In [8]:
bedrock_agent = BedrockAgentsRunnable(
    agent_id="JQNNUTIFGE", 
    agent_alias_id="ADSXXXOCVK",
    client=bedrock_agent
)

In [9]:
chat_history = []

In [50]:
@tool
@traceable
def interact_with_agent(input_query, chat_history):
    """Interact with the agent and store chat history. Return the response.
    When communicating with the main agent always call this tool with the exception of looking for the video."""
    result = bedrock_agent.invoke(
        {
            "input": input_query,
            "chat_history": chat_history,
        }
    )
    print(result)
    chat_history.append(input_query)
    chat_history.append("Assistant: " + result.return_values['output'])
    return result


# Define a tool that uses the DuckDuckGoSearchResults tool
@tool
@traceable
def search_tool(input_data):
    """Searches for the youtube videos explaining the problem."""
    return search.run(input_data)


# # Human interaction tool (could be a simple input function or more complex UI)
# @tool
# def human_review(input_data):
#     print(f"Review the following data: {input_data}")
#     decision = input("Enter your decision (approve/modify): ")
#     if decision.lower() == 'modify':
#         modified_data = input("Enter the modified data: ")
#         return modified_data
#     return input_data

system_message = """You are a Teacher of preparation for the coding interview. You are preparing a student for the coding interview. 
When the student is communicating with you always call the interact_with_agent_tool with only one exception - when the student wants to find a video
with the explanation of the problem. In this case, call the search_tool.
When the user asks you to give the problem in specific topic - call interact_with_agent_tool with the user request as an input.
If the user asks you for a video - call the search_tool with the user request and problem as an input."""

model = ChatOpenAI(model="gpt-4o", openai_api_key=conf.open_ai_key)
app = create_react_agent(model, [interact_with_agent, search_tool], state_modifier=system_message,)

query = "Please explain linked lists to me"
message_history = []
@traceable
def pipeline(query, message_history):
    
    if len(message_history) > 1:
        history = [message['messages'] for message in message_history[-1:]]
    elif message_history == []:
        history = []
    else:
        history = message_history['messages']
    return app.invoke({"messages": history + [("human", query)]})

messages = pipeline(query, message_history)
message_history = messages


return_values={'output': 'A linked list is a linear data structure where each element (called a node) contains a value and a reference (or pointer) to the next node in the sequence. The first node is called the head, and the last node points to null to mark the end of the list.\n\nEach node in a linked list has two parts:\n1. Data: The value or information that the node stores.\n2. Next: A pointer or reference to the next node in the list.\n\nThe basic structure of a node in a singly linked list can be represented as:\n\nclass Node:\n    def __init__(self, data):\n        self.data = data\n        self.next = None\n\nSome key operations on linked lists:\n- Insertion: Adding a new node at the beginning, end, or a specific position in the list.\n- Deletion: Removing a node from the beginning, end, or a specific position in the list.\n- Traversal: Visiting each node in the list to perform some operation.\n\nLinked lists have some advantages over arrays, such as dynamic size and efficient 

In [51]:
messages

{'messages': [HumanMessage(content='Please explain linked lists to me', id='406464c1-a8f5-411c-aa2f-9d74260c1ce5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2gVcpaq5CkBD1QeQjpObCSw', 'function': {'arguments': '{"input_query":"Please explain linked lists to me","chat_history":[]}', 'name': 'interact_with_agent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 230, 'total_tokens': 256}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d525fab9-74c2-403a-bd07-ee8c91cd1fcc-0', tool_calls=[{'name': 'interact_with_agent', 'args': {'input_query': 'Please explain linked lists to me', 'chat_history': ['Please explain linked lists to me', 'Assistant: A linked list is a linear data structure where each element (called a node) contains a value and a reference (or pointer) to the next node in the sequence. The first node is called the

In [57]:
messages2 = pipeline('Give me a problem in linked lists', message_history)


return_values={'output': 'Could you please provide me with another linked list problem for a coding interview preparation? I would like the problem statement, examples of how it should work, and the necessary code definitions like the structure of a linked list node.'} log='Could you please provide me with another linked list problem for a coding interview preparation? I would like the problem statement, examples of how it should work, and the necessary code definitions like the structure of a linked list node.' session_id='c80e0373-0b21-4a97-85bd-91cf15ed4dc0'


In [58]:
message_history += messages2

In [59]:
messages2

{'messages': [HumanMessage(content='Please explain linked lists to me', id='406464c1-a8f5-411c-aa2f-9d74260c1ce5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2gVcpaq5CkBD1QeQjpObCSw', 'function': {'arguments': '{"input_query":"Please explain linked lists to me","chat_history":[]}', 'name': 'interact_with_agent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 230, 'total_tokens': 256}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d525fab9-74c2-403a-bd07-ee8c91cd1fcc-0', tool_calls=[{'name': 'interact_with_agent', 'args': {'input_query': 'Please explain linked lists to me', 'chat_history': ['Please explain linked lists to me', 'Assistant: A linked list is a linear data structure where each element (called a node) contains a value and a reference (or pointer) to the next node in the sequence. The first node is called the

In [55]:
messages3 = pipeline('Give a hint', message_history)
message_history += messages3
messages3

return_values={'output': 'The key idea is to maintain three pointers: prev (initially None), curr (initially head), and next (initially curr.next). Then we iterate through the list, updating the next pointer of the current node to point to the previous node, and moving the prev and curr pointers one step forward. At the end, we return prev as the new head of the reversed list.'} log='The key idea is to maintain three pointers: prev (initially None), curr (initially head), and next (initially curr.next). Then we iterate through the list, updating the next pointer of the current node to point to the previous node, and moving the prev and curr pointers one step forward. At the end, we return prev as the new head of the reversed list.' session_id='924c1ed4-11c1-43d5-b6e1-56d20d43c182'


{'messages': [HumanMessage(content='Please explain linked lists to me', id='406464c1-a8f5-411c-aa2f-9d74260c1ce5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2gVcpaq5CkBD1QeQjpObCSw', 'function': {'arguments': '{"input_query":"Please explain linked lists to me","chat_history":[]}', 'name': 'interact_with_agent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 230, 'total_tokens': 256}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d525fab9-74c2-403a-bd07-ee8c91cd1fcc-0', tool_calls=[{'name': 'interact_with_agent', 'args': {'input_query': 'Please explain linked lists to me', 'chat_history': ['Please explain linked lists to me', 'Assistant: A linked list is a linear data structure where each element (called a node) contains a value and a reference (or pointer) to the next node in the sequence. The first node is called the

In [56]:
messages4 = pipeline('Give me a video which explains it', message_history)
message_history += messages4
messages4

{'messages': [HumanMessage(content='Please explain linked lists to me', id='406464c1-a8f5-411c-aa2f-9d74260c1ce5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2gVcpaq5CkBD1QeQjpObCSw', 'function': {'arguments': '{"input_query":"Please explain linked lists to me","chat_history":[]}', 'name': 'interact_with_agent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 230, 'total_tokens': 256}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d525fab9-74c2-403a-bd07-ee8c91cd1fcc-0', tool_calls=[{'name': 'interact_with_agent', 'args': {'input_query': 'Please explain linked lists to me', 'chat_history': ['Please explain linked lists to me', 'Assistant: A linked list is a linear data structure where each element (called a node) contains a value and a reference (or pointer) to the next node in the sequence. The first node is called the